## CNN - Convolution Neural Network

Learn the features from mesh data.

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*RIBWK55dcDJa-zI_dFPDnw.png)

![](https://saturncloud.io/images/blog/a-cnn-sequence-to-classify-handwritten-digits.webp)

![](https://upload.wikimedia.org/wikipedia/commons/9/90/CNN-filter-animation-1.gif?20230201202141)

![](https://developers.google.com/static/machine-learning/practica/image-classification/images/maxpool_animation.gif?hl=zh-tw)

![](https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/73_blog_image_1.png)

In [ ]:
import numpy as np
arr = np.loadtxt("sample_data/mnist_train_small.csv", delimiter=",", dtype=int)
X = arr[1:, 1:]
Y = arr[1:, 0]
print(f"X: {len(X)}\n", X, "\n", X[0])
print(f"Y: {len(Y)}\n", Y, "\n", Y[0])
from PIL import Image
img = X[0]
print(Y[0])
Image.fromarray(img.reshape(28, 28).astype(np.uint8)).resize((140, 140)).show()
def oneHot(y):
    labels = np.unique(y)
    labels.sort()
    label_dict = dict()
    for value in labels:
        key = np.where(labels == value)[0][0]
        label_dict[key] = str(value)
        y = np.where(y == value, key, y)
    y = y.astype(int)
    # y = np.eye(len(np.unique(y)))[y].astype(int)
    return y, label_dict

Y, label_dict = oneHot(Y)
X_train, X_test = X[:18000, :], X[18000:, :]
Y_train, Y_test = Y[:18000], Y[18000:]
print(Y_train)
print(label_dict)

In [ ]:
import torch
X_train_tensor = torch.from_numpy(X_train.astype("float32") / 255)
X_test_tensor = torch.from_numpy(X_test.astype("float32") / 255)
Y_train_tensor = torch.from_numpy(Y_train.astype("float32")).type(torch.LongTensor)
Y_test_tensor = torch.from_numpy(Y_test.astype("float32")).type(torch.LongTensor)
print(X_train_tensor)
print(Y_train_tensor)
# pytorch train and test TensorDataset
train = torch.utils.data.TensorDataset(X_train_tensor, Y_train_tensor)
test = torch.utils.data.TensorDataset(X_test_tensor, Y_test_tensor)

![](https://saturncloud.io/images/blog/a-cnn-sequence-to-classify-handwritten-digits.webp)

In [ ]:
# hyperparameters
lr = 1e-3
epochs = 50
batch_size = 16
# pytorch DataLoader
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)

# define CNN Model
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Convolution 1 , input_shape=(1,28,28)
        self.cnn1 = torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0) #output_shape=(16,24,24)
        self.relu1 = torch.nn.ReLU() # activation
        # Max pool 1
        self.maxpool1 = torch.nn.MaxPool2d(kernel_size=2) #output_shape=(16,12,12)
        # Convolution 2
        self.cnn2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0) #output_shape=(32,8,8)
        self.relu2 = torch.nn.ReLU() # activation
        # Max pool 2
        self.maxpool2 = torch.nn.MaxPool2d(kernel_size=2) #output_shape=(32,4,4)
        # Fully connected 1 ,#input_shape=(32*4*4)
        self.fc1 = torch.nn.Linear(32 * 4 * 4, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        # Max pool 1
        out = self.maxpool1(out)
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        # Max pool 2 
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        # Linear function (readout)
        out = self.fc1(out)
        return out

# training
model = CNN()
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # optimize all cnn parameters
loss_func = torch.nn.CrossEntropyLoss()   # the target label is not one-hotted
input_shape = (-1,1,28,28)

In [ ]:
def fit_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, test_loader):
    # Traning the Model
    #history-like list for store loss & acc value
    training_loss = []
    training_accuracy = []
    validation_loss = []
    validation_accuracy = []
    for epoch in range(num_epochs):
        #training model & store loss & acc / epoch
        correct_train = 0
        total_train = 0
        for i, (images, labels) in enumerate(train_loader):
            # 1.Define variables
            train = torch.autograd.Variable(images.view(input_shape))
            labels = torch.autograd.Variable(labels)
            # 2.Clear gradients
            optimizer.zero_grad()
            # 3.Forward propagation
            outputs = model(train)
            # 4.Calculate softmax and cross entropy loss
            train_loss = loss_func(outputs, labels)
            # 5.Calculate gradients
            train_loss.backward()
            # 6.Update parameters
            optimizer.step()
            # 7.Get predictions from the maximum value
            predicted = torch.max(outputs.data, 1)[1]
            # 8.Total number of labels
            total_train += len(labels)
            # 9.Total correct predictions
            correct_train += (predicted == labels).float().sum()
        #10.store val_acc / epoch
        train_accuracy = 100 * correct_train / float(total_train)
        training_accuracy.append(train_accuracy)
        # 11.store loss / epoch
        training_loss.append(train_loss.data)

        #evaluate model & store loss & acc / epoch
        correct_test = 0
        total_test = 0
        for images, labels in test_loader:
            # 1.Define variables
            test = torch.autograd.Variable(images.view(input_shape))
            # 2.Forward propagation
            outputs = model(test)
            # 3.Calculate softmax and cross entropy loss
            val_loss = loss_func(outputs, labels)
            # 4.Get predictions from the maximum value
            predicted = torch.max(outputs.data, 1)[1]
            # 5.Total number of labels
            total_test += len(labels)
            # 6.Total correct predictions
            correct_test += (predicted == labels).float().sum()
        #6.store val_acc / epoch
        val_accuracy = 100 * correct_test / float(total_test)
        validation_accuracy.append(val_accuracy)
        # 11.store val_loss / epoch
        validation_loss.append(val_loss.data)
        print("Epoch {: 5d}/{}\t- loss: {:.4f} - acc: {:.4f} - val_loss: {:.4f} - val_acc: {:.4f}".format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
    return training_loss, training_accuracy, validation_loss, validation_accuracy

In [ ]:
training_loss, training_accuracy, validation_loss, validation_accuracy = fit_model(model, loss_func, optimizer, input_shape, epochs, train_loader, test_loader)

In [ ]:
# visualization
import matplotlib.pyplot as plt
plt.plot(range(num_epochs), training_loss, 'b-', label='Training_loss')
plt.plot(range(num_epochs), validation_loss, 'g-', label='validation_loss')
plt.title('Training & Validation loss')
plt.xlabel('Number of epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(range(num_epochs), training_accuracy, 'b-', label='Training_accuracy')
plt.plot(range(num_epochs), validation_accuracy, 'g-', label='Validation_accuracy')
plt.title('Training & Validation accuracy')
plt.xlabel('Number of epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Reference
[PyTorch - CNN](https://hackmd.io/@lido2370/SJMPbNnKN?type=view)